In [ ]:
# !pip install -q imageio
# !pip install -q git+https://github.com/tensorflow/docs
# !pip install tensorflow-hub

## GPU 구동

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "0"

## cache 초기화

In [2]:
import torch

torch.cuda.empty_cache()

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

import imageio
from IPython.display import HTML, display

# MoveNet

In [20]:
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Maps bones to a matplotlib color name.
KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}
# KEYPOINT_EDGE_INDS_TO_COLOR = {
#     (0, 1): (255, 0, 255),
#     (0, 2): (0, 255, 255),
#     (1, 3): (255, 0, 255),
#     (2, 4): (0, 255, 255),
#     (0, 5): (255, 0, 255),
#     (0, 6): (0, 255, 255),
#     (5, 7): (255, 0, 255),
#     (7, 9): (255, 0, 255),
#     (6, 8): (0, 255, 255),
#     (8, 10): (0, 255, 255),
#     (5, 6): (0, 255, 0),
#     (5, 11): (255, 0, 255),
#     (6, 12): (0, 255, 255),
#     (11, 12): (0, 255, 0),
#     (11, 13): (255, 0, 255),
#     (13, 15): (255, 0, 255),
#     (12, 14): (0, 255, 255),
#     (14, 16): (0, 255, 255)
# }

def _keypoints_and_edges_for_display(keypoints_with_scores,
                                     height,
                                     width,
                                     keypoint_threshold=0.11):
  """Returns high confidence keypoints and edges for visualization.

  Args:
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    height: height of the image in pixels.
    width: width of the image in pixels.
    keypoint_threshold: minimum confidence score for a keypoint to be
      visualized.

  Returns:
    A (keypoints_xy, edges_xy, edge_colors) containing:
      * the coordinates of all keypoints of all detected entities;
      * the coordinates of all skeleton edges of all detected entities;
      * the colors in which the edges should be plotted.
  """
  keypoints_all = []
  keypoint_edges_all = []
  edge_colors = []
  num_instances, _, _, _ = keypoints_with_scores.shape
  for idx in range(num_instances):
    kpts_x = keypoints_with_scores[0, idx, :, 1]
    kpts_y = keypoints_with_scores[0, idx, :, 0]
    kpts_scores = keypoints_with_scores[0, idx, :, 2]
    kpts_absolute_xy = np.stack(
        [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
    kpts_above_thresh_absolute = kpts_absolute_xy[
        kpts_scores > keypoint_threshold, :]
    keypoints_all.append(kpts_above_thresh_absolute)

    for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
      if (kpts_scores[edge_pair[0]] > keypoint_threshold and
          kpts_scores[edge_pair[1]] > keypoint_threshold):
        x_start = kpts_absolute_xy[edge_pair[0], 0]
        y_start = kpts_absolute_xy[edge_pair[0], 1]
        x_end = kpts_absolute_xy[edge_pair[1], 0]
        y_end = kpts_absolute_xy[edge_pair[1], 1]
        line_seg = np.array([[x_start, y_start], [x_end, y_end]])
        keypoint_edges_all.append(line_seg)
        edge_colors.append(color)
  if keypoints_all:
    keypoints_xy = np.concatenate(keypoints_all, axis=0)
  else:
    keypoints_xy = np.zeros((0, 17, 2))

  if keypoint_edges_all:
    edges_xy = np.stack(keypoint_edges_all, axis=0)
  else:
    edges_xy = np.zeros((0, 2, 2))
  return keypoints_xy, edges_xy, edge_colors


def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)  
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot

# def draw_keypoints(frame, keypoints, confidence_threshold=0.3):
#   """OpenCV를 사용하여 keypoints와 스켈레톤 그리기"""
#   y, x, c = frame.shape
#   shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

#   # 각 keypoint 그리기
#   for kp in shaped:
#     ky, kx, kp_conf = kp
#     if kp_conf > confidence_threshold:
#       cv2.circle(frame, (int(kx), int(ky)), 4, (0, 255, 0), -1)

#   # keypoint들 연결 선 그리기
#   for edge, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
#     p1, p2 = edge
#     y1, x1, c1 = shaped[p1]
#     y2, x2, c2 = shaped[p2]
#     if c1 > confidence_threshold and c2 > confidence_threshold:
#       cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)



def to_gif(images, duration):
  """Converts image sequence (4D numpy array) to gif."""
  imageio.mimsave('./animation.gif', images, duration=duration)
  return embed.embed_file('./animation.gif')

def progress(value, max=100):
  return HTML("""
      
          {value}
      
  """.format(value=value, max=max))

In [8]:

model_name = "movenet_lightning" #@param ["movenet_lightning", "movenet_thunder", "movenet_lightning_f16.tflite", "movenet_thunder_f16.tflite", "movenet_lightning_int8.tflite", "movenet_thunder_int8.tflite"]

if "tflite" in model_name:
  if "movenet_lightning_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
    input_size = 256
  elif "movenet_lightning_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores

In [9]:
#@title Cropping Algorithm

# Confidence score to determine whether a keypoint prediction is reliable.
MIN_CROP_KEYPOINT_SCORE = 0.2

def init_crop_region(image_height, image_width):
  """Defines the default crop region.

  The function provides the initial crop region (pads the full image from both
  sides to make it a square image) when the algorithm cannot reliably determine
  the crop region from the previous frame.
  """
  if image_width > image_height:
    box_height = image_width / image_height
    box_width = 1.0
    y_min = (image_height / 2 - image_width / 2) / image_height
    x_min = 0.0
  else:
    box_height = 1.0
    box_width = image_height / image_width
    y_min = 0.0
    x_min = (image_width / 2 - image_height / 2) / image_width

  return {
    'y_min': y_min,
    'x_min': x_min,
    'y_max': y_min + box_height,
    'x_max': x_min + box_width,
    'height': box_height,
    'width': box_width
  }

def torso_visible(keypoints):
  """Checks whether there are enough torso keypoints.

  This function checks whether the model is confident at predicting one of the
  shoulders/hips which is required to determine a good crop region.
  """
  return ((keypoints[0, 0, KEYPOINT_DICT['left_hip'], 2] >
           MIN_CROP_KEYPOINT_SCORE or
          keypoints[0, 0, KEYPOINT_DICT['right_hip'], 2] >
           MIN_CROP_KEYPOINT_SCORE) and
          (keypoints[0, 0, KEYPOINT_DICT['left_shoulder'], 2] >
           MIN_CROP_KEYPOINT_SCORE or
          keypoints[0, 0, KEYPOINT_DICT['right_shoulder'], 2] >
           MIN_CROP_KEYPOINT_SCORE))

def determine_torso_and_body_range(
    keypoints, target_keypoints, center_y, center_x):
  """Calculates the maximum distance from each keypoints to the center location.

  The function returns the maximum distances from the two sets of keypoints:
  full 17 keypoints and 4 torso keypoints. The returned information will be
  used to determine the crop size. See determineCropRegion for more detail.
  """
  torso_joints = ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']
  max_torso_yrange = 0.0
  max_torso_xrange = 0.0
  for joint in torso_joints:
    dist_y = abs(center_y - target_keypoints[joint][0])
    dist_x = abs(center_x - target_keypoints[joint][1])
    if dist_y > max_torso_yrange:
      max_torso_yrange = dist_y
    if dist_x > max_torso_xrange:
      max_torso_xrange = dist_x

  max_body_yrange = 0.0
  max_body_xrange = 0.0
  for joint in KEYPOINT_DICT.keys():
    if keypoints[0, 0, KEYPOINT_DICT[joint], 2] < MIN_CROP_KEYPOINT_SCORE:
      continue
    dist_y = abs(center_y - target_keypoints[joint][0]);
    dist_x = abs(center_x - target_keypoints[joint][1]);
    if dist_y > max_body_yrange:
      max_body_yrange = dist_y

    if dist_x > max_body_xrange:
      max_body_xrange = dist_x

  return [max_torso_yrange, max_torso_xrange, max_body_yrange, max_body_xrange]

def determine_crop_region(
      keypoints, image_height,
      image_width):
  """Determines the region to crop the image for the model to run inference on.

  The algorithm uses the detected joints from the previous frame to estimate
  the square region that encloses the full body of the target person and
  centers at the midpoint of two hip joints. The crop size is determined by
  the distances between each joints and the center point.
  When the model is not confident with the four torso joint predictions, the
  function returns a default crop which is the full image padded to square.
  """
  target_keypoints = {}
  for joint in KEYPOINT_DICT.keys():
    target_keypoints[joint] = [
      keypoints[0, 0, KEYPOINT_DICT[joint], 0] * image_height,
      keypoints[0, 0, KEYPOINT_DICT[joint], 1] * image_width
    ]

  if torso_visible(keypoints):
    center_y = (target_keypoints['left_hip'][0] +
                target_keypoints['right_hip'][0]) / 2;
    center_x = (target_keypoints['left_hip'][1] +
                target_keypoints['right_hip'][1]) / 2;

    (max_torso_yrange, max_torso_xrange,
      max_body_yrange, max_body_xrange) = determine_torso_and_body_range(
          keypoints, target_keypoints, center_y, center_x)

    crop_length_half = np.amax(
        [max_torso_xrange * 1.9, max_torso_yrange * 1.9,
          max_body_yrange * 1.2, max_body_xrange * 1.2])

    tmp = np.array(
        [center_x, image_width - center_x, center_y, image_height - center_y])
    crop_length_half = np.amin(
        [crop_length_half, np.amax(tmp)]);

    crop_corner = [center_y - crop_length_half, center_x - crop_length_half];

    if crop_length_half > max(image_width, image_height) / 2:
      return init_crop_region(image_height, image_width)
    else:
      crop_length = crop_length_half * 2;
      return {
        'y_min': crop_corner[0] / image_height,
        'x_min': crop_corner[1] / image_width,
        'y_max': (crop_corner[0] + crop_length) / image_height,
        'x_max': (crop_corner[1] + crop_length) / image_width,
        'height': (crop_corner[0] + crop_length) / image_height -
            crop_corner[0] / image_height,
        'width': (crop_corner[1] + crop_length) / image_width -
            crop_corner[1] / image_width
      }
  else:
    return init_crop_region(image_height, image_width)

def crop_and_resize(image, crop_region, crop_size):
  """Crops and resize the image to prepare for the model input."""
  boxes=[[crop_region['y_min'], crop_region['x_min'],
          crop_region['y_max'], crop_region['x_max']]]
  output_image = tf.image.crop_and_resize(
      image, box_indices=[0], boxes=boxes, crop_size=crop_size)
  return output_image

def run_inference(movenet, image, crop_region, crop_size):
  """Runs model inferece on the cropped region.

  The function runs the model inference on the cropped region and updates the
  model output to the original image coordinate system.
  """
  image_height, image_width, _ = image.shape
  input_image = crop_and_resize(
    tf.expand_dims(image, axis=0), crop_region, crop_size=crop_size)
  # Run model inference.
  keypoints_with_scores = movenet(input_image)
  # Update the coordinates.
  for idx in range(17):
    keypoints_with_scores[0, 0, idx, 0] = (
        crop_region['y_min'] * image_height +
        crop_region['height'] * image_height *
        keypoints_with_scores[0, 0, idx, 0]) / image_height
    keypoints_with_scores[0, 0, idx, 1] = (
        crop_region['x_min'] * image_width +
        crop_region['width'] * image_width *
        keypoints_with_scores[0, 0, idx, 1]) / image_width
  return keypoints_with_scores

In [10]:
# !wget -q -O dance.gif https://github.com/tensorflow/tfjs-models/raw/master/pose-detection/assets/dance_input.gif


In [10]:
def extract_features(keypoints_with_scores):
    """관절 좌표를 기반으로 특징을 추출"""
    keypoints = keypoints_with_scores[0, 0, :, :2]  # 각 프레임의 관절 좌표만 추출
    keypoints = keypoints.flatten()
    return keypoints

### Video

In [40]:
# import numpy as np
# def prepare_dataset(videos):
#     X = []
#     y = []
#     for video, label in videos:
#         cap = cv2.VideoCapture(video)
#         while cap.isOpened():
#             ret, frame = cap.read()
#             if not ret:
#                 break
#             # 이미지를 movenet 입력 사이즈에 맞게 조정
#             frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             resized_frame = cv2.resize(frame_rgb, (input_size, input_size))
#             input_image = tf.image.resize_with_pad(np.expand_dims(resized_frame, axis=0), input_size, input_size)
#             input_image = tf.cast(input_image, dtype=tf.int32)

#             # Movenet으로 관절 추출
#             keypoints_with_scores = movenet(input_image)
#             features = extract_features(keypoints_with_scores)
#             X.append(features)
#             y.append(label) # 0: ADL, 1: Fall
#         cap.release()
#     return np.array(X), np.array(y)
            

### Video_image

In [11]:
# prepare_dataset 함수를 이미지 파일도 처리할 수 있도록 수정
def prepare_dataset(videos_or_images):
    X = []
    y = []
    for file_path, label in videos_or_images:
        if file_path.endswith('.mp4'):
            # 비디오 파일 처리
            cap = cv2.VideoCapture(file_path)
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                # 프레임 크기 축소
                frame = cv2.resize(frame, (640, 480))
                
                # Movenet을 통해 keypoints 추출
                input_image = tf.image.resize_with_pad(np.expand_dims(frame, axis=0), input_size, input_size)
                input_image = tf.cast(input_image, dtype=tf.int32)
                keypoints_with_scores = movenet(input_image)
                
                # 특징 추출
                features = extract_features(keypoints_with_scores)
                X.append(features)
                y.append(label)
            cap.release()
        elif file_path.endswith(('.jpg', '.png', '.jpeg')):
            # 이미지 파일 처리
            image = cv2.imread(file_path)
            image = cv2.resize(image, (640, 480))
            
            # Movenet을 통해 keypoints 추출
            input_image = tf.image.resize_with_pad(np.expand_dims(image, axis=0), input_size, input_size)
            input_image = tf.cast(input_image, dtype=tf.int32)
            keypoints_with_scores = movenet(input_image)
            
            # 특징 추출
            features = extract_features(keypoints_with_scores)
            X.append(features)
            y.append(label)
    return np.array(X), np.array(y)


## 학습 데이터

In [14]:
# 학습 및 테스트 데이터 분할
from sklearn.model_selection import train_test_split
videos_or_images = [("./data/pose_est/ADL/01.mp4", 0),
          ("./data/pose_est/ADL/02.mp4", 0),
          ("./data/pose_est/ADL/03.mp4", 0),
          ("./data/pose_est/ADL/04.mp4", 0),
          ("./data/pose_est/ADL/05.mp4", 0),
          ("./data/pose_est/ADL/06.mp4", 0),
          ("./data/pose_est/ADL/07.mp4", 0),
          ("./data/pose_est/ADL/08.mp4", 0),
          ("./data/pose_est/ADL/09.mp4", 0),
          ("./data/pose_est/ADL/10.mp4", 0),
          ("./data/pose_est/ADL/11.mp4", 0),
          ("./data/pose_est/ADL/12.mp4", 0),
          ("./data/pose_est/ADL/13.mp4", 0),
          ("./data/pose_est/ADL/14.mp4", 0),
          ("./data/pose_est/ADL/15.mp4", 0),
          ("./data/pose_est/ADL/16.mp4", 0),
          ("./data/pose_est/ADL/not fallen001.jpg", 0),
          ("./data/pose_est/ADL/not fallen002.jpg", 0),
          ("./data/pose_est/ADL/not fallen003.jpg", 0),
          ("./data/pose_est/ADL/not fallen004.jpg", 0),
          ("./data/pose_est/ADL/not fallen005.jpg", 0),
          ("./data/pose_est/ADL/not fallen006.jpg", 0),
          ("./data/pose_est/ADL/not fallen007.jpg", 0),
          ("./data/pose_est/ADL/not fallen008.jpg", 0),
          ("./data/pose_est/ADL/not fallen009.jpg", 0),
          ("./data/pose_est/ADL/not fallen010.jpg", 0),
          ("./data/pose_est/ADL/not fallen011.jpg", 0),
          ("./data/pose_est/ADL/not fallen012.jpg", 0),
          ("./data/pose_est/ADL/not fallen013.jpg", 0),
          ("./data/pose_est/ADL/not fallen014.jpg", 0),
          ("./data/pose_est/ADL/not fallen015.jpg", 0),
          ("./data/pose_est/ADL/not fallen016.jpg", 0),
          ("./data/pose_est/ADL/not fallen017.jpg", 0),
          ("./data/pose_est/ADL/not fallen018.jpg", 0),
          ("./data/pose_est/ADL/not fallen019.jpg", 0),
          ("./data/pose_est/ADL/not fallen020.jpg", 0),
          ("./data/pose_est/ADL/not fallen021.jpg", 0),
          ("./data/pose_est/ADL/not fallen022.jpg", 0),
          ("./data/pose_est/ADL/not fallen023.jpg", 0),
          ("./data/pose_est/ADL/not fallen024.jpg", 0),
          ("./data/pose_est/ADL/not fallen025.jpg", 0),
          ("./data/pose_est/ADL/not fallen026.jpg", 0),
          ("./data/pose_est/ADL/not fallen027.jpg", 0),
          ("./data/pose_est/ADL/not fallen028.jpg", 0),
          ("./data/pose_est/ADL/not fallen029.jpg", 0),
          ("./data/pose_est/ADL/not fallen030.jpg", 0),
          ("./data/pose_est/ADL/not fallen040.jpg", 0),
          ("./data/pose_est/ADL/not fallen041.jpg", 0),
          ("./data/pose_est/ADL/not fallen042.jpg", 0),
          ("./data/pose_est/ADL/not fallen043.jpg", 0),
          ("./data/pose_est/ADL/not fallen044.jpg", 0),
          ("./data/pose_est/ADL/not fallen045.jpg", 0),
          ("./data/pose_est/ADL/not fallen046.jpg", 0),
          ("./data/pose_est/ADL/not fallen047.jpg", 0),
          ("./data/pose_est/ADL/not fallen048.jpg", 0),
          ("./data/pose_est/ADL/not fallen049.jpg", 0),
          ("./data/pose_est/ADL/not fallen050.jpg", 0),
          ("./data/pose_est/ADL/not fallen051.jpg", 0),
          ("./data/pose_est/ADL/not fallen052.jpg", 0),
          ("./data/pose_est/ADL/not fallen053.jpg", 0),
          ("./data/pose_est/ADL/not fallen054.jpg", 0),
          ("./data/pose_est/ADL/not fallen055.jpg", 0),
          ("./data/pose_est/ADL/not fallen056.jpg", 0),
          ("./data/pose_est/ADL/not fallen057.jpg", 0),
          ("./data/pose_est/ADL/not fallen058.jpg", 0),
          ("./data/pose_est/ADL/not fallen059.jpg", 0),
          ("./data/pose_est/ADL/not fallen060.jpg", 0),
          ("./data/pose_est/ADL/not fallen061.jpg", 0),
          ("./data/pose_est/ADL/not fallen062.jpg", 0),
          ("./data/pose_est/ADL/not fallen063.jpg", 0),
          ("./data/pose_est/ADL/not fallen064.jpg", 0),
          ("./data/pose_est/ADL/not fallen065.jpg", 0),
          ("./data/pose_est/ADL/not fallen066.jpg", 0),
          ("./data/pose_est/ADL/not fallen067.jpg", 0),
          ("./data/pose_est/ADL/not fallen068.jpg", 0),
          ("./data/pose_est/ADL/not fallen069.jpg", 0),
          ("./data/pose_est/ADL/not fallen070.jpg", 0),
          ("./data/pose_est/ADL/not fallen071.jpg", 0),
          ("./data/pose_est/ADL/not fallen072.jpg", 0),
          ("./data/pose_est/ADL/not fallen073.jpg", 0),
          ("./data/pose_est/ADL/not fallen074.jpg", 0),
          ("./data/pose_est/ADL/not fallen075.jpg", 0),
          ("./data/pose_est/ADL/not fallen076.jpg", 0),
          ("./data/pose_est/ADL/not fallen077.jpg", 0),
          ("./data/pose_est/ADL/not fallen078.jpg", 0),
          ("./data/pose_est/ADL/not fallen079.jpg", 0),
          ("./data/pose_est/ADL/not fallen080.jpg", 0),
          ("./data/pose_est/ADL/not fallen081.jpg", 0),
          ("./data/pose_est/ADL/not fallen082.jpg", 0),
          ("./data/pose_est/ADL/not fallen083.jpg", 0),
          ("./data/pose_est/ADL/not fallen084.jpg", 0),
          ("./data/pose_est/ADL/not fallen085.jpg", 0),
          ("./data/pose_est/ADL/not fallen086.jpg", 0),
          ("./data/pose_est/ADL/not fallen087.jpg", 0),
          ("./data/pose_est/ADL/not fallen088.jpg", 0),
          ("./data/pose_est/ADL/not fallen089.jpg", 0),
          ("./data/pose_est/ADL/not fallen090.jpg", 0),
          ("./data/pose_est/ADL/not fallen091.jpg", 0),
          ("./data/pose_est/ADL/not fallen092.jpg", 0),
          ("./data/pose_est/ADL/not fallen093.jpg", 0),
          ("./data/pose_est/ADL/not fallen094.jpg", 0),
          ("./data/pose_est/ADL/not fallen095.jpg", 0),
          ("./data/pose_est/ADL/not fallen096.jpg", 0),
          ("./data/pose_est/ADL/not fallen097.jpg", 0),
          ("./data/pose_est/ADL/not fallen098.jpg", 0),
          ("./data/pose_est/ADL/not fallen099.jpg", 0),
          ("./data/pose_est/ADL/not fallen100.jpg", 0),
          ("./data/pose_est/ADL/not fallen101.jpg", 0),
          ("./data/pose_est/ADL/not fallen102.jpg", 0),
          ("./data/pose_est/ADL/not fallen103.jpg", 0),
          ("./data/pose_est/ADL/not fallen104.jpg", 0),
          ("./data/pose_est/ADL/not fallen105.jpg", 0),
          ("./data/pose_est/ADL/not fallen106.jpg", 0),
          ("./data/pose_est/ADL/not fallen107.jpg", 0),
          ("./data/pose_est/ADL/not fallen108.jpg", 0),
          ("./data/pose_est/ADL/not fallen109.jpg", 0),
          ("./data/pose_est/ADL/not fallen110.jpg", 0),
          ("./data/pose_est/ADL/not fallen111.jpg", 0),
          ("./data/pose_est/ADL/not fallen112.jpg", 0),
          ("./data/pose_est/ADL/not fallen113.jpg", 0),
          ("./data/pose_est/ADL/not fallen114.jpg", 0),
          ("./data/pose_est/ADL/not fallen115.jpg", 0),
          ("./data/pose_est/ADL/not fallen116.jpg", 0),
          ("./data/pose_est/ADL/not fallen117.jpg", 0),
          ("./data/pose_est/ADL/not fallen118.jpg", 0),
          ("./data/pose_est/ADL/not fallen119.jpg", 0),
          ("./data/pose_est/ADL/not fallen120.jpg", 0),
          ("./data/pose_est/ADL/not fallen121.jpg", 0),
          ("./data/pose_est/ADL/not fallen122.jpg", 0),
          ("./data/pose_est/ADL/not fallen123.jpg", 0),
          ("./data/pose_est/ADL/not fallen124.jpg", 0),
          ("./data/pose_est/ADL/not fallen125.jpg", 0),
          ("./data/pose_est/ADL/not fallen126.jpg", 0),
          ("./data/pose_est/ADL/not fallen127.jpg", 0),
          ("./data/pose_est/ADL/not fallen128.jpg", 0),
          ("./data/pose_est/ADL/not fallen129.jpg", 0),
          ("./data/pose_est/ADL/not fallen130.jpg", 0),
          ("./data/pose_est/ADL/not fallen131.jpg", 0),
          ("./data/pose_est/ADL/not fallen132.jpg", 0),
          ("./data/pose_est/ADL/not fallen133.jpg", 0),
          ("./data/pose_est/ADL/not fallen134.jpg", 0),
          ("./data/pose_est/ADL/not fallen135.jpg", 0),
          ("./data/pose_est/ADL/not fallen136.jpg", 0),
          ("./data/pose_est/ADL/not fallen137.jpg", 0),
          ("./data/pose_est/ADL/not fallen138.jpg", 0),
          ("./data/pose_est/ADL/not fallen139.jpg", 0),
          ("./data/pose_est/ADL/not fallen140.jpg", 0),
          ("./data/pose_est/ADL/not fallen141.jpg", 0),
          ("./data/pose_est/ADL/not fallen142.jpg", 0),
          ("./data/pose_est/ADL/not fallen143.jpg", 0),
          ("./data/pose_est/ADL/not fallen144.jpg", 0),
          ("./data/pose_est/ADL/not fallen145.jpg", 0),
          ("./data/pose_est/ADL/not fallen146.jpg", 0),
          ("./data/pose_est/ADL/not fallen147.jpg", 0),
          ("./data/pose_est/ADL/not fallen148.jpg", 0),
          ("./data/pose_est/ADL/not fallen149.jpg", 0),
          ("./data/pose_est/ADL/not fallen150.jpg", 0),
          ("./data/pose_est/ADL/not fallen151.jpg", 0),
          ("./data/pose_est/ADL/not fallen152.jpg", 0),
          ("./data/pose_est/ADL/not fallen153.jpg", 0),
          ("./data/pose_est/ADL/not fallen154.jpg", 0),
          ("./data/pose_est/ADL/not fallen155.jpg", 0),
          ("./data/pose_est/ADL/not fallen156.jpg", 0),
          ("./data/pose_est/ADL/not fallen157.jpg", 0),
          ("./data/pose_est/ADL/not fallen158.jpg", 0),
          ("./data/pose_est/ADL/not fallen159.jpg", 0),
          ("./data/pose_est/ADL/not fallen160.jpg", 0),
          ("./data/pose_est/ADL/not fallen161.jpg", 0),
          ("./data/pose_est/ADL/not fallen162.jpg", 0),
          ("./data/pose_est/ADL/not fallen163.jpg", 0),
          ("./data/pose_est/ADL/not fallen164.jpg", 0),
          ("./data/pose_est/ADL/not fallen165.jpg", 0),
          ("./data/pose_est/ADL/not fallen166.jpg", 0),

           ("./data/pose_est/Fall/01.mp4", 1),
           ("./data/pose_est/Fall/02.mp4", 1),
           ("./data/pose_est/Fall/03.mp4", 1),
           ("./data/pose_est/Fall/04.mp4", 1),
           ("./data/pose_est/Fall/05.mp4", 1),
           ("./data/pose_est/Fall/06.mp4", 1),
           ("./data/pose_est/Fall/07.mp4", 1),
           ("./data/pose_est/Fall/08.mp4", 1),
           ("./data/pose_est/Fall/09.mp4", 1),
           ("./data/pose_est/Fall/10.mp4", 1),
           ("./data/pose_est/Fall/11.mp4", 1),
           ("./data/pose_est/Fall/12.mp4", 1),
           ("./data/pose_est/Fall/13.mp4", 1),
           ("./data/pose_est/Fall/14.mp4", 1),
           ("./data/pose_est/Fall/15.mp4", 1),
           ("./data/pose_est/Fall/16.mp4", 1),
           ("./data/pose_est/Fall/fall001.jpg", 1),
           ("./data/pose_est/Fall/fall002.jpg", 1),
           ("./data/pose_est/Fall/fall003.jpg", 1),
           ("./data/pose_est/Fall/fall004.jpg", 1),
           ("./data/pose_est/Fall/fall005.jpg", 1),
           ("./data/pose_est/Fall/fall006.jpg", 1),
           ("./data/pose_est/Fall/fall007.jpg", 1),
           ("./data/pose_est/Fall/fall008.jpg", 1),
           ("./data/pose_est/Fall/fall009.jpg", 1),
           ("./data/pose_est/Fall/fall010.jpg", 1),
           ("./data/pose_est/Fall/fall011.jpg", 1),
           ("./data/pose_est/Fall/fall012.jpg", 1),
           ("./data/pose_est/Fall/fall013.jpg", 1),
           ("./data/pose_est/Fall/fall014.jpg", 1),
           ("./data/pose_est/Fall/fall015.jpg", 1),
           ("./data/pose_est/Fall/fall016.jpg", 1),
           ("./data/pose_est/Fall/fall017.jpg", 1),
           ("./data/pose_est/Fall/fall018.jpg", 1),
           ("./data/pose_est/Fall/fall019.jpg", 1),
           ("./data/pose_est/Fall/fall020.jpg", 1),
           ("./data/pose_est/Fall/fall021.jpg", 1),
           ("./data/pose_est/Fall/fall022.jpg", 1),
           ("./data/pose_est/Fall/fall023.jpg", 1),
           ("./data/pose_est/Fall/fall024.jpg", 1),
           ("./data/pose_est/Fall/fall025.jpg", 1),
           ("./data/pose_est/Fall/fall026.jpg", 1),
           ("./data/pose_est/Fall/fall027.jpg", 1),
           ("./data/pose_est/Fall/fall028.jpg", 1),
           ("./data/pose_est/Fall/fall029.jpg", 1),
           ("./data/pose_est/Fall/fall030.jpg", 1),
           ("./data/pose_est/Fall/fall031.jpg", 1),
           ("./data/pose_est/Fall/fall032.jpg", 1),
           ("./data/pose_est/Fall/fall033.jpg", 1),
           ("./data/pose_est/Fall/fall034.jpg", 1),
           ("./data/pose_est/Fall/fall035.jpg", 1),
           ("./data/pose_est/Fall/fall036.jpg", 1),
           ("./data/pose_est/Fall/fall037.jpg", 1),
           ("./data/pose_est/Fall/fall038.jpg", 1),
           ("./data/pose_est/Fall/fall039.jpg", 1),
           ("./data/pose_est/Fall/fall040.jpg", 1),
           ("./data/pose_est/Fall/fall041.jpg", 1),
           ("./data/pose_est/Fall/fall042.jpg", 1),
           ("./data/pose_est/Fall/fall043.jpg", 1),
           ("./data/pose_est/Fall/fall044.jpg", 1),
           ("./data/pose_est/Fall/fall045.jpg", 1),
           ("./data/pose_est/Fall/fall046.jpg", 1),
           ("./data/pose_est/Fall/fall047.jpg", 1),
           ("./data/pose_est/Fall/fall048.jpg", 1),
           ("./data/pose_est/Fall/fall049.jpg", 1),
           ("./data/pose_est/Fall/fall050.jpg", 1),
           ("./data/pose_est/Fall/fall051.jpg", 1),
           ("./data/pose_est/Fall/fall052.jpg", 1),
           ("./data/pose_est/Fall/fall053.jpg", 1),
           ("./data/pose_est/Fall/fall054.jpg", 1),
           ("./data/pose_est/Fall/fall055.jpg", 1),
           ("./data/pose_est/Fall/fall056.jpg", 1),
           ("./data/pose_est/Fall/fall057.jpg", 1),
           ("./data/pose_est/Fall/fall058.jpg", 1),
           ("./data/pose_est/Fall/fall059.jpg", 1),
           ("./data/pose_est/Fall/fall060.jpg", 1),
           ("./data/pose_est/Fall/fall061.jpg", 1),
           ("./data/pose_est/Fall/fall062.jpg", 1),
           ("./data/pose_est/Fall/fall063.jpg", 1),
           ("./data/pose_est/Fall/fall064.jpg", 1),
           ("./data/pose_est/Fall/fall065.jpg", 1),
           ("./data/pose_est/Fall/fall066.jpg", 1),
           ("./data/pose_est/Fall/fall067.jpg", 1),
           ("./data/pose_est/Fall/fall068.jpg", 1),
           ("./data/pose_est/Fall/fall069.jpg", 1),
           ("./data/pose_est/Fall/fall070.jpg", 1),
           ("./data/pose_est/Fall/fall071.jpg", 1),
           ("./data/pose_est/Fall/fall072.jpg", 1),
           ("./data/pose_est/Fall/fall073.jpg", 1),
           ("./data/pose_est/Fall/fall074.jpg", 1),
           ("./data/pose_est/Fall/fall075.jpg", 1),
           ("./data/pose_est/Fall/fall076.jpg", 1),
           ("./data/pose_est/Fall/fall077.jpg", 1),
           ("./data/pose_est/Fall/fall078.jpg", 1),
           ("./data/pose_est/Fall/fall079.jpg", 1),
           ("./data/pose_est/Fall/fall080.jpg", 1),
           ("./data/pose_est/Fall/fall081.jpg", 1),
           ("./data/pose_est/Fall/fall082.jpg", 1),
           ("./data/pose_est/Fall/fall083.jpg", 1),
           ("./data/pose_est/Fall/fall084.jpg", 1),
           ("./data/pose_est/Fall/fall085.jpg", 1),
           ("./data/pose_est/Fall/fall086.jpg", 1),
           ("./data/pose_est/Fall/fall087.jpg", 1),
           ("./data/pose_est/Fall/fall088.jpg", 1),
           ("./data/pose_est/Fall/fall089.jpg", 1),
           ("./data/pose_est/Fall/fall090.jpg", 1),
           ("./data/pose_est/Fall/fall091.jpg", 1),
           ("./data/pose_est/Fall/fall092.jpg", 1),
           ("./data/pose_est/Fall/fall093.jpg", 1),
           ("./data/pose_est/Fall/fall094.jpg", 1),
           ("./data/pose_est/Fall/fall095.jpg", 1),
           ("./data/pose_est/Fall/fall096.jpg", 1),
           ("./data/pose_est/Fall/fall097.jpg", 1),
           ("./data/pose_est/Fall/fall098.jpg", 1),
           ("./data/pose_est/Fall/fall099.jpg", 1),
           ("./data/pose_est/Fall/fall100.jpg", 1),
           ("./data/pose_est/Fall/fall101.jpg", 1),
           ("./data/pose_est/Fall/fall102.jpg", 1),
           ("./data/pose_est/Fall/fall103.jpg", 1),
           ("./data/pose_est/Fall/fall104.jpg", 1),
           ("./data/pose_est/Fall/fall105.jpg", 1),
           ("./data/pose_est/Fall/fall106.jpg", 1),
           ("./data/pose_est/Fall/fall107.jpg", 1),
           ("./data/pose_est/Fall/fall108.jpg", 1),
           ("./data/pose_est/Fall/fall109.jpg", 1),
           ("./data/pose_est/Fall/fall110.jpg", 1),
           ("./data/pose_est/Fall/fall111.jpg", 1),
           ("./data/pose_est/Fall/fall112.jpg", 1),
           ("./data/pose_est/Fall/fall113.jpg", 1),
           ("./data/pose_est/Fall/fall114.jpg", 1),
           ("./data/pose_est/Fall/fall115.jpg", 1),
           ("./data/pose_est/Fall/fall116.jpg", 1),
           ("./data/pose_est/Fall/fall117.jpg", 1),
           ("./data/pose_est/Fall/fall118.jpg", 1),
           ("./data/pose_est/Fall/fall119.jpg", 1),
           ("./data/pose_est/Fall/fall120.jpg", 1),
           ("./data/pose_est/Fall/fall121.jpg", 1),
           ("./data/pose_est/Fall/fall122.jpg", 1),
           ("./data/pose_est/Fall/fall123.jpg", 1),
           ("./data/pose_est/Fall/fall124.jpg", 1),
           ("./data/pose_est/Fall/fall125.jpg", 1),
           ("./data/pose_est/Fall/fall126.jpg", 1),
           ("./data/pose_est/Fall/fall127.jpg", 1),
           ("./data/pose_est/Fall/fall128.jpg", 1),
           ("./data/pose_est/Fall/fall129.jpg", 1),
           ("./data/pose_est/Fall/fall130.jpg", 1),
           ("./data/pose_est/Fall/fall131.jpg", 1),
           ("./data/pose_est/Fall/fall132.jpg", 1),
           ("./data/pose_est/Fall/fall133.jpg", 1),
           ("./data/pose_est/Fall/fall134.jpg", 1),
           ("./data/pose_est/Fall/fall135.jpg", 1),
           ("./data/pose_est/Fall/fall136.jpg", 1),
           ("./data/pose_est/Fall/fall137.jpg", 1),
           ("./data/pose_est/Fall/fall138.jpg", 1),
           ("./data/pose_est/Fall/fall139.jpg", 1),
           ("./data/pose_est/Fall/fall140.jpg", 1),
           ("./data/pose_est/Fall/fall141.jpg", 1),
           ("./data/pose_est/Fall/fall142.jpg", 1),
           ("./data/pose_est/Fall/fall143.jpg", 1),
           ("./data/pose_est/Fall/fall144.jpg", 1),
           ("./data/pose_est/Fall/fall145.jpg", 1),
           ("./data/pose_est/Fall/fall146.jpg", 1),
           ("./data/pose_est/Fall/fall147.jpg", 1),
           ("./data/pose_est/Fall/fall148.jpg", 1),
           ("./data/pose_est/Fall/fall149.jpg", 1),
           ("./data/pose_est/Fall/fall150.jpg", 1),
           ("./data/pose_est/Fall/fall151.jpg", 1),
           ("./data/pose_est/Fall/fall152.jpg", 1),
           ("./data/pose_est/Fall/fall153.jpg", 1),
           ("./data/pose_est/Fall/fall154.jpg", 1),
           ("./data/pose_est/Fall/fall155.jpg", 1),
           ("./data/pose_est/Fall/fall156.jpg", 1),
           ("./data/pose_est/Fall/fall157.jpg", 1),
           ("./data/pose_est/Fall/fall158.jpg", 1),
           ("./data/pose_est/Fall/fall159.jpg", 1),
           ("./data/pose_est/Fall/fall160.jpg", 1),
           ("./data/pose_est/Fall/fall161.jpg", 1),
           ("./data/pose_est/Fall/fall162.jpg", 1),
           ("./data/pose_est/Fall/fall163.jpg", 1),
           ("./data/pose_est/Fall/fall164.jpg", 1),
           ("./data/pose_est/Fall/fall165.jpg", 1),
           ("./data/pose_est/Fall/fall166.jpg", 1)
           ]
X, y = prepare_dataset(videos_or_images)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [23]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', max_iter=200)
model.fit(X_train, y_train)

LogisticRegression(max_iter=200)

In [24]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.8436507936507937


# 녹화 비디오 탐지 

In [ ]:
def real_time_fall_detection(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        image_height, image_width, _ = frame.shape
        crop_region = init_crop_region(image_height, image_width)
        keypoints_with_scores = run_inference(movenet, frame, crop_region, crop_size=(input_size, input_size))
        # 특징 추출
        features = extract_features(keypoints_with_scores)

        # 추락 여부 추측
        is_fall = model.predict([features])[0]
        if is_fall:
            # print("추락 감지!")
            cv2.putText(frame, "Fall Detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # keypoints 시각화
        frame_with_keypoints = draw_prediction_on_image(frame, keypoints_with_scores)


        # 결과 프레임 보여주기
        cv2.imshow("Fall Detection", frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

real_time_fall_detection("./data/pose_est/Fall/12.mp4")

# 실시간 탐지

In [ ]:
def real_time_fall_detection(video_source=0):
    cap = cv2.VideoCapture(video_source)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        image_height, image_width, _ = frame.shape
        crop_region = init_crop_region(image_height, image_width)
        keypoints_with_scores = run_inference(movenet, frame, crop_region, crop_size=(input_size, input_size))
        # 특징 추출
        features = extract_features(keypoints_with_scores)

        # 추락 여부 추측
        is_fall = model.predict([features])[0]
        if is_fall:
            # print("추락 감지!")
            cv2.putText(frame, "Fall Detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # keypoints 시각화
        frame_with_keypoints = draw_prediction_on_image(frame, keypoints_with_scores)


        # 결과 프레임 보여주기
        cv2.imshow("Fall Detection", frame_with_keypoints)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

real_time_fall_detection(0)

In [65]:
# real_time_fall_detection 수정: OpenCV 시각화 추가 및 속도 최적화
def real_time_fall_detection(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # 프레임 크기 축소
        frame = cv2.resize(frame, (640, 480))  # 프레임 크기를 640x480으로 축소
        
        # Movenet을 통해 keypoints 추론
        input_image = tf.image.resize_with_pad(np.expand_dims(frame, axis=0), input_size, input_size)
        input_image = tf.cast(input_image, dtype=tf.int32)
        keypoints_with_scores = movenet(input_image)
        # 특징 추출
        features = extract_features(keypoints_with_scores)
        # 추락 여부 추측
        is_fall = model.predict([features])[0]
        if is_fall:
            print("추락 감지!")
            cv2.putText(frame, "Fall Detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # 특징 추출 및 추락 감지 (여기서는 생략, 대신 시각화)
        draw_keypoints(frame, keypoints_with_scores)

        # 결과 프레임 보여주기
        cv2.imshow("Fall Detection", frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 예시 실행
real_time_fall_detection("./data/pose_est/Fall/15.mp4")

추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!

In [67]:
# real_time_fall_detection 수정: OpenCV 시각화 추가 및 속도 최적화
def real_time_fall_detection(video_source=0):
    cap = cv2.VideoCapture(video_source)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # 프레임 크기 축소
        frame = cv2.resize(frame, (640, 480))  # 프레임 크기를 640x480으로 축소
        
        # Movenet을 통해 keypoints 추론
        input_image = tf.image.resize_with_pad(np.expand_dims(frame, axis=0), input_size, input_size)
        input_image = tf.cast(input_image, dtype=tf.int32)
        keypoints_with_scores = movenet(input_image)
        # 특징 추출
        features = extract_features(keypoints_with_scores)
        # 추락 여부 추측
        is_fall = model.predict([features])[0]
        if is_fall:
            print("추락 감지!")
            cv2.putText(frame, "Fall Detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # 특징 추출 및 추락 감지 (여기서는 생략, 대신 시각화)
        draw_keypoints(frame, keypoints_with_scores)

        # 결과 프레임 보여주기
        cv2.imshow("Fall Detection", frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 예시 실행
real_time_fall_detection(2)

추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!
추락 감지!

In [34]:

# # Load the input image.
# image_path = 'dance.gif'
# image = tf.io.read_file(image_path)
# image = tf.image.decode_gif(image)

In [ ]:
# # Load the input image.
# num_frames, image_height, image_width, _ = image.shape
# crop_region = init_crop_region(image_height, image_width)

# output_images = []
# bar = display(progress(0, num_frames-1), display_id=True)
# for frame_idx in range(num_frames):
#   keypoints_with_scores = run_inference(
#       movenet, image[frame_idx, :, :, :], crop_region,
#       crop_size=[input_size, input_size])
#   output_images.append(draw_prediction_on_image(
#       image[frame_idx, :, :, :].numpy().astype(np.int32),
#       keypoints_with_scores, crop_region=None,
#       close_figure=True, output_image_height=300))
#   crop_region = determine_crop_region(
#       keypoints_with_scores, image_height, image_width)
#   bar.update(progress(frame_idx, num_frames-1))

# # Prepare gif visualization.
# output = np.stack(output_images, axis=0)
# to_gif(output, duration=100)